# Spherical Harmonics from Scratch

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def calculate_K(l: int, m: int) -> float:
    return np.sqrt(
        (2 * l + 1)
        / (4 * np.math.pi)
        * np.math.factorial(l - m)
        / np.math.factorial(l + m)
    )

In [7]:
def calculate_P(l: int, m: int, x: float) -> float:
    pmm = 1.0  # value of polynomial when l == m
    if m > 0:
        somx2 = np.sqrt((1.0 - x) * (1.0 + x))
        fact = 1.0  # factor, increasing by 2 at each iter
        for i in range(1, m + 1):
            pmm *= -fact * somx2
            fact += 2.0
    if l == m:
        return pmm

    pmmp1 = x * (2.0 * m + 1.0) * pmm  # value of polynomial when l == m + 1
    if l == m + 1:
        return pmmp1

    pll = 0.0  # value of polynomial when l > m + 1
    for ll in range(m + 2, l + 1):
        pll = ((2.0 * ll - 1.0) * x * pmmp1 - (ll + m - 1.0) * pmm) / (ll - m)
        pmm = pmmp1
        pmmp1 = pll

    return pll

In [8]:
def calculate_SH(l: int, m: int, theta: float, phi: float) -> float:
    if m == 0:
        return calculate_K(l, m) * calculate_P(l, m, np.cos(theta))

    if m > 0:
        return (
            np.sqrt(2.0)
            * calculate_K(l, m)
            * np.cos(m * phi)
            * calculate_P(l, m, np.cos(theta))
        )

    return (
        np.sqrt(2.0)
        * calculate_K(l, -m)
        * np.sin(-m * phi)
        * calculate_P(l, -m, np.cos(theta))
    )

## Signal Encoding using SH

In [9]:
def encode_sh_coefficients(l: int, m: int) -> float:
    STEPS_PHI = 200
    STEPS_THETA = 100
    d_phi = 2 * np.math.pi / STEPS_PHI
    d_theta = np.math.pi / STEPS_THETA

    coeff = 0.0
    for i in range(STEPS_PHI):
        phi = i * d_phi
        for j in range(STEPS_THETA):
            theta = (0.5 + j) * d_theta
            coeff += calculate_SH(l, m, theta, phi) * np.sin(theta) * d_theta * d_phi

    return coeff